In [1]:
import pandas as pd
from utils import preprocessing
from pathlib import Path

In [2]:
# Define channels
columns = {
    "imu": {
        "wx": True,
        "wy": True,
        "wz": True,
        "ax": True,
        "ay": True,
        "az": True,
    },
    "pro": {
        "velL": True,
        "velR": True,
        "curL": True,
        "curR": True,
    },
}
summary = pd.DataFrame({"columns": pd.Series(columns)})

csv_dir = Path("data")

In [3]:
# Get recordings
data = preprocessing.get_recordings(csv_dir, summary)
data["imu"].shape

(41025, 9)

In [4]:
# Unmerged columns
unmerged = ["terrain", "run_idx"]

# Highest sampling frequency
hf_sensor = summary["sampling_freq"].idxmax()
hf = summary["sampling_freq"].max()
# Other sensors are low frequency
lf_sensors = tuple(sens for sens in data.keys() if sens != hf_sensor)

merged = []

hf_data = data[hf_sensor]
terrains = sorted(hf_data.terrain.unique().tolist())
terr = terrains[0]
terrdat = {sens: sdata[sdata.terrain == terr] for sens, sdata in data.items()}

hf_terr = terrdat[hf_sensor]

exps = {
    sens: sdata[sdata.run_idx == 1].copy().reset_index(drop=True)
    for sens, sdata in terrdat.items()
}

hf_exp = exps[hf_sensor]


In [5]:
hf_exp.head()

,time,wx,wy,wz,ax,ay,az,terrain,run_idx
0,0.00,-0.048889,0.131503,-0.010670,0.515528,-0.352803,10.279377,CONCRETE,1
1,0.02,-0.005499,0.021856,-0.020985,0.345918,-0.216189,9.333001,CONCRETE,1
2,0.04,0.007669,-0.028620,-0.017898,0.073693,0.379493,9.478169,CONCRETE,1
3,0.06,0.003153,-0.029832,-0.025182,-0.283525,0.879968,10.009389,CONCRETE,1
4,0.08,-0.094245,-0.030998,-0.030244,-0.696063,0.726217,10.632477,CONCRETE,1


In [6]:
lf_exp = exps[lf_sensors[0]]
cols = [c for c in lf_exp.columns if c not in unmerged]
lf_exp = lf_exp[cols].copy()
lf_exp.head()

,time,velL,velR,curL,curR
0,0.000000,0.467364,0.460056,6.11,8.47
1,0.066667,0.559204,0.559976,3.88,5.58
2,0.133333,0.300700,0.301682,1.51,2.04
3,0.200000,0.559321,0.559132,0.00,2.04
4,0.266667,0.581990,0.582390,2.04,2.96


In [14]:
merged = hf_exp.merge(lf_exp, how="outer", on="time")
merged[cols] = merged[cols].ffill()
merged = merged.dropna().reset_index(drop=True)


In [23]:
hf_exp.shape

(1018, 9)

In [17]:
merged.run_idx.astype(int)

0       1
1       1
2       1
3       1
4       1
       ..
1013    1
1014    1
1015    1
1016    1
1017    1
Name: run_idx, Length: 1018, dtype: int64

In [19]:
hf_exp.run_idx, merged.run_idx

(0       1
 1       1
 2       1
 3       1
 4       1
        ..
 1013    1
 1014    1
 1015    1
 1016    1
 1017    1
 Name: run_idx, Length: 1018, dtype: int64,
 0       1.0
 1       1.0
 2       1.0
 3       1.0
 4       1.0
        ... 
 1013    1.0
 1014    1.0
 1015    1.0
 1016    1.0
 1017    1.0
 Name: run_idx, Length: 1018, dtype: float64)

In [21]:
from utils.transforms import merge_dfs
merge_dfs(exps, hf_sensor).shape

(1018, 13)

In [22]:
merged.shape

(1018, 13)